In [11]:
import import_ipynb
import enthalpy_functions
import pandas as pd 
import matplotlib.pyplot as plt
#plt.rcParams['savefig.dpi'] = 1200
#plt.rcParams["figure.figsize"] = [10,10]
import numpy as np
import numpy_financial as npf #for calculating the NPV and IRR 

In [12]:
## Electrolyser financial calculations


def electrolyser_npv(tax_rate,interest_rate,electricity_cost,
             iron_ore_cost,emission_cost,carbon_steel_price,O2_price,el_spec,h2_investment_2020):
    # Metallization rates considered for the DRI reduction shaft furnace
    import import_ipynb
    import enthalpy_functions
    mol_weight_fe2o3=159.688 #in grams
    alpha=0.94 
    # Percentage of hematite in the raw material. 
    fe2O3_pure=0.95 
    # percentage of iron in one mole of iron ore 
    Fe_O_ratio=((2*mol_weight_fe)/mol_weight_fe2o3)
    # kg of steel produced in the DRI-EAF system (1 ton)
    m3=1000
    # Weight of total iron ore fed to the DRI-EAF system
    m1=m3/(fe2O3_pure*Fe_O_ratio*alpha)
    T1=298 #pellet temperature
    # Depends on the slag content , metallization rate 
    m2_feo=(m1*fe2O3_pure*(1-alpha)*Fe_O_ratio) 
    #print("FeO percentage :",m4_feo)
    # Percentage of silica in the ore pellets
    SIO2_per=0.03 
    #Percentage of alumina in the ore pellets
    Al2O3_per=0.02 
    m1_sio2=(m1*SIO2_per)# SiO2 weight in ton
    m1_al2o3=(m1*Al2O3_per) # weight in ton
    m2_fe=(m1-(m1_sio2+m1_al2o3+m2_feo))*Fe_O_ratio


    ## Stocichiometric calculations


    # to produce 1 mole of iron 2/3 rd mole of hydrogen are required
    H2_per_mol=3/2 
    # Weight of H2 for producing one mole of Fe in gms
    H2_weight=(H2_per_mol)*mol_weight_H2 
    # Number of moles in 1 ton of Fe
    mol_per_ton_fe=(1000000)/mol_weight_fe
    # Weight of hydrogen in kg to reduce 1 ton of Fe2O3
    m4_stoichiometric=(H2_weight*mol_per_ton_fe)/1000
     #  Extra h2 considered
    lambda_h2=1.2
    m4=m4_stoichiometric*lambda_h2


    h2_prod_yr=200000000 #in kg per year 
    # Plant is operational 95% of the time in an year 
    operating_hours=365*24*0.95 
    h2_prod_hr=h2_prod_yr/operating_hours
    # Calculating the actual amount of hydrogen required for production of one ton of steel
    #Excess hydrogen fed in the dri shaft furnace for complete reduction of iron ore 

    h2_per_ton_actual=lambda_h2*m4_stoichiometric
    m4=h2_per_ton_actual # mass flow rate of hydrogen entering the shaft furnace
    steel_per_hr=h2_prod_hr/h2_per_ton_actual


    # Global variable declaration for the base case
    plant_life=20
    # Design parameter
    lang_factor=3
    steel_prod_yr=steel_per_hr*operating_hours # 2.5 million tonnes per year 

    # 
    O2_sold=0.6 # it is assumed that 60% of the oxygen is sold

    ## Enthalpy of hydrogen entering the shaft furnace

    dri_temp=900
    T4=(900+273.15) # k Entry of hydrogen at 900 C 



    if T4<1000:
        h4=(H2_enthalpy_1(T4)*m4*1000) #kj/kg
        h4_kwh=(h4/3600)
    elif T4>1000:
        h4=(H2_enthalpy_2(T4)*m4*1000) #kj/kg 
        h4_kwh=(h4/3600)


    ## Mass flow rate of waste gases from the shaft furnace 

    # Mass flow rate of water in the waste stream per ton of steel production

    water_tls=((3*mol_weight_H2O)/(2*mol_weight_fe))*1000

    ## Calculation of the hydrogen in the waste stream 
    m5_h2=(m4_stoichiometric*(lambda_h2-1)) #Kg/tls
    m5_h2o=water_tls
    m5=(m5_h2+m5_h2o)
    # The value varies with input flow rate of hydrogen inside the shaft furnace


    ## Exhaust stream enthalpy

    """Temperature of the exhaust stream could vary according to the input
    temperature and flow rate of the hydrogen stream.
    Plant data from Syngas based DRI shaft furnace show that the temperature could vary from 275 C to 400 C.
    Higher temperatures are witnessed when the content of CO in the Syngas is higher.
    The reduction reaction between hematite and CO is exothermic.
    On the other hand , hyrogen reduction of hematite is endothermic.
    Assumed temperature of 300 C for the enthalpy calculation."""


    # K Temperature of the exhaust gas leaving the reduction shaft furnace
    T5=573

    ### Energy contained in water stream


    h5_h2o=(m5_h2o*H2O_enthalpy(T5)*1000) #kJ/tls
    h5_h2=(H2_enthalpy_1(T5)*m5_h2*1000) #kJ/tls
    h5=(h5_h2o+h5_h2)
    h5_kwh=(h5/3600)
    h5_kwh

    ## Reaction enthalpy calculation
    #The reaction enthalpy has been taken as 99.5 kJ/mol based on stoichiometric calculations.

    h_activation=35 # kJ/mol activation enthalpy
    h_endothermic=99.5 # reaction enthalpy kJ/mol
    #Adding the activation enthalpy to the reaction enthalpy
    # value in kJ/mol
    h_reaction=h_activation+h_endothermic
    # Amount of heat to be supplied in kJ for production of 1 ton of molten steel
    h_reac=(h_reaction*m1*1000*fe2O3_pure)/mol_weight_fe2o3
    #print("Reaction enthalpy in kJ/ton of liquid steel:",h_reac) 
    #print("Reaction enthalpy in KWh/ton of liquid steel:",h_reac/3600)

    ## Energy contained in the solid stream exiting the DRI

    """Enthalpy of the residual FeO in the slag stream : https://webbook.nist.gov/cgi/cbook.cgi?ID=C1345251&Mask=2"""

    # The constants are used for a temperature range of 298-1650 K 
    # Exit temperature of the FeO stream contained in the solid stream exiting the reactor

    T2=873


    h2=(fe_enthalpy(T2)*m2_fe+sio2_enthalpy(T2)*m1_sio2+
        Al2O3_enthalpy(T2)*m1_al2o3+feo_enthalpy(T2)*m2_feo)*1000

    h2_kwh=h2/3600

    h2_kwh

    ## Energy balance of the shaft furnace


    Q_dri=((h4)-(h5+h2+h_reac))/3600



    ## Heat recuperator calculation

    T5=573 # K

    T6=393 # k to ensure that there is no condensation in the heat exchanger
    T7=343 #k Water is assumed to condense at 70 C
    T8=343
    ratio_burner=0.80
    m8=m4


    m6_h2o=m5_h2o
    m6_h2=m5_h2
    m7=m6_h2o


    h6_h2o=(m6_h2o*H2O_enthalpy(T6)*1000) #kJ/tls
    h6_h2=(H2_enthalpy_1(T6)*m6_h2*1000) #kJ/tls
    h6=(h6_h2o+h6_h2)
    h6_kwh=h6/3600
    h8=(H2_enthalpy_1(T8)*m8*1000)
    h8_kwh=(h8/3600)


    ###  Recupertor energy balance 
    h11=((h5-h6)+h8)

    ## Calculate h8

    ## Calculate h9


    T9=413# K  

    m9=m8

    h9=(H2_enthalpy_1(T9)*m9*1000)
    h9_kwh=(h9/3600)


    ## Calculate eta_rec 

    eta_rec=((h9-h8)/(h5-h6))
    ## Heat required in the heater 

    Q_heater=(h4-h9)/3600 #kj/tls

    eta_el_heater=0.6

    el_heater=(Q_heater/eta_el_heater)



    ## Electrolyser : Mass and energy flow

    #el_spec=50 # KWh/kgh2
    #KWH/Kg of hydrogen 
    water_spec=11 #11 litres of water is required for production of 1 kg of hydrogen 

    el_elec=(m4*el_spec) # el_spec should be in KWh/kgH2
    water_total=(m4*water_spec)
    extra_h2o=(water_total-m7)




    ## Electric arc furnace mass and energy balance 

    ## input and tap temperatures
    T2=873# # Temperature input to the EAF at 600 C
    T3=1923 # Tap temperature of the molten steel 1800 C 
    "Reference : Development of mass and energy balance model and its application for HBI charged EAF"

    ## Energy required to melt Fe in the EAF

    # Reference : NIST https://webbook.nist.gov/cgi/cbook.cgi?ID=C7439896&Mask=2&Type=JANAFS&Table=on
    Hfe_melting=247 # Melting enthalpy of steel per ton in Kj/kg
    #https://materialsdata.nist.gov/bitstream/handle/11115/166/Thermophysical%20Properties.pdf?sequence=3
    Hfe_T2=fe_enthalpy(T2) # Enthalpy of DRI at the input 
    Hfe_T3=fe_enthalpy_2(T3) # Enthalpy of molten steel at the output
    h3=(Hfe_T3-Hfe_T2)*m2_fe*1000+m2_fe*Hfe_melting#kj/kg
    h3_kwh=h3/3600 #kwh/tls



    eta_el=0.6 # Efficiency of the transformer, arc 
    #, heat transfer, cooling losses, waste gas stream taken into consideration
    # The efficincy is considered lower to account for the loss of energy from the
    #scrap stream, the use of slag formers etc
    el_eaf=(h3_kwh)/eta_el
    #print ("Electrical energy input in kWh/ton of liquid steel",el_eaf)

    # Specific energy consumption of HDRI-EAF system


    EL_total=(el_eaf+el_elec+el_heater)

    EL_total_MWh=EL_total/1000




    # Specific CO2 emissions from the HDRI-EAF system

    emission_factor=0.413 # corresponds to the emissions from EU grid 
    EAF_co2=0.050 #ton/tls
    cao_emissions=0.056 #tco2/tls
    co2_eaf_electrode=0.0070 #tco2/tls
    pellet_production=0.12#tco2/tls
    direct_emission=EAF_co2+cao_emissions+co2_eaf_electrode+pellet_production
    indirect_emission_eaf=(el_eaf*emission_factor)/1000
    indirect_emission_el=(el_elec*emission_factor)/1000
    total_emission=direct_emission+indirect_emission_eaf+indirect_emission_el #tco2/tls

    # input parameters for NPV calculation

    # Plant is operational 95% of the time in an year 
    operating_hours=365*24*0.95 
    steel_per_hr=steel_prod_yr/operating_hours
    h2_per_hour_kg=steel_per_hr*(m4)
    h2_per_second_kg=(h2_per_hour_kg)/3600
    lhv_h2=120.1 #MJ/kg
    h2_capacity_MW=h2_per_second_kg*lhv_h2
    
    electrolyser_efficiency=0.67
    el_capacity_mwel=h2_capacity_MW/ electrolyser_efficiency 

    #print(h2_capacity_MW)
    """# Capital investment values taken from Hydrogen generation in Europe: Overview of key costs and benefits
    July 2020"""
    stack_lifetime=90000 # hours 
    stack_replacement_year=stack_lifetime/operating_hours
    stack_replacement_number=1
    Euro_dollar_conversion=1.18
    ## USD 600 /KW

    h2_investment_MW_h2=(h2_investment_2020*Euro_dollar_conversion)# million dollars per MW H2 out 

    electrolyser_cost=h2_investment_MW_h2*h2_capacity_MW
    # only the stack is replaced, which is 60% of the total el capital cost 
    # it is assumed that by the time stacks are replaced the cost of electrolyzers would 
    #have fallen to 0.45 Million Euro/MW of H2
    # replacement is considered only once for the plant as its assumed that the lifetime of 
    #next generation stacks would be more than 15 years 
    h2_investment_2030=0.45
    #0.6 represents 60 % of the stack needs to be replaced after lifetime 
    replacement_cost=electrolyser_cost*0.6*h2_investment_2030*np.round(stack_replacement_number)
    total_electrolyser_cost=(electrolyser_cost+replacement_cost)
    #### Economic parameters that could be included in the article
    EAF_cost_per_ton_yr=140
    # The value includes installation costs 
    #https://iea-etsap.org/E-TechDS/PDF/I02-Iron&Steel-GS-AD-gct.pdf
    EAF_total_cost=(EAF_cost_per_ton_yr*steel_prod_yr)/10**6


    DRI_cost_per_ton_yr=240 #USD/ton/per year
    # Cost of new DRI plant with 2 MT/annum =5.5 million euros or 6 million USD
     # A reduction in cost is considered as larger plants have smaller costs.
    # The plant costs include all the costs including ISBL, OSBL, area development etc 
    # no Lang factor multiplication is required for such a plant
    #https://www.voestalpine.com/group/en/media/press-releases/2013-07-04-
     #voestalpine-entrusts-construction-of-the-direct-reduction-plant-in-texas-to-siemens-and-midrex/
    DRI_total_cost=(DRI_cost_per_ton_yr*steel_prod_yr)/10**6
    #### Pressure swing adsorber 
     #To account for the balance of plant equipment like the condenser, heat exchangers, heaters etc.
     # Million USD 
    
    total_capital_cost=((electrolyser_cost)+EAF_total_cost+DRI_total_cost)*lang_factor
    ## Emission cost


    EAF_co2=0.050 #ton/tls from exhaust gases 
    cao_emissions=0.056 #tco2/tls calcination of caco3
    co2_eaf_electrode=0.0070 #tco2/tls use of electrodes
    pellet_production=0.12#tco2/tls # upstream emissions from pellet production
    total_emission=((EAF_co2)+(EL_total_MWh*emission_factor)) #tco2/tls
    # The steel plant does not need to pay for the emissions from the grid. 
    total_emission_cost=(((EAF_co2+cao_emissions+co2_eaf_electrode+pellet_production)*emission_cost*steel_prod_yr)/10**6)
    #Emission cost


    #operational cost 
    eaf_op_cost_tls=32 #t/yr of dri
    dri_op_cost_tls=13 #t/yr of dri
    #ng_cost=((ng_price/21)*natural_gas_kg)# converting price/MMBTU to price/kg
    iron_ore_tls=m1/1000 # Converting iron ore required from kg to tonnes
    iron_ore_cost_tls=iron_ore_tls*iron_ore_cost # Includes the pelletization cost
    electricity_cost_tls=EL_total_MWh*electricity_cost 
    operational_cost_annual=((iron_ore_cost_tls+electricity_cost_tls+
                                  eaf_op_cost_tls+dri_op_cost_tls)*steel_prod_yr)/10**6


    #Maintenance cost
    maintenance_cost_percentage=0.05 
    labour_cost_tls=40 #USD/tls
    # 5% of the capital cost

    maintenance_cost_yr=((maintenance_cost_percentage*total_capital_cost))
    labour_cost_yr=((labour_cost_tls*steel_prod_yr)/10**6)

    depreciation_yr=total_capital_cost/plant_life

     # Revenue
    # USD/ton
    O2_produced_tls=m4*8
    total_O2_produced=(O2_produced_tls*steel_prod_yr)/1000#in tons

    O2_revenue=(total_O2_produced*O2_price*O2_sold)
    total_revenue=((carbon_steel_price*steel_prod_yr)+O2_revenue)/10**6


    ## NPV and IRR calculations 
    years=np.arange(0,plant_life,1)
    Years=np.round(years,0)
    production_steel=np.repeat(steel_prod_yr/10**6,plant_life).tolist()
    #production_hydrogen=np.repeat(h2_prod_yr/1000000,plant_life).tolist()  

    capital_cost_yr=np.repeat(0,plant_life).tolist()
    capital_cost_yr[0]=((total_capital_cost-replacement_cost)/3)
    capital_cost_yr[1]=(total_capital_cost-replacement_cost)*(2/3)
    capital_cost_yr[11]=replacement_cost
    operational_cost=np.repeat(operational_cost_annual,plant_life).tolist()
    maintenance_cost=np.repeat(maintenance_cost_yr,plant_life).tolist()
    labour_cost=np.repeat(labour_cost_yr,plant_life).tolist()
    emission_cost=np.repeat(total_emission_cost,plant_life).tolist()
    depreciation_cost=np.repeat(depreciation_yr,plant_life).tolist()
    total_revenue=np.repeat(total_revenue,plant_life).tolist()




    for i in range(2):
        production_steel[i]=0
        operational_cost[i]=0
        labour_cost[i]=0
        maintenance_cost[i]=0
        emission_cost[i]=0
        depreciation_cost[i]=0
        total_revenue[i]=0
    # tax calculation



        tax_cost=[(total_revenue[i]-(capital_cost_yr[i]+operational_cost[i]+        
                                labour_cost[i]+maintenance_cost[i]
                                +emission_cost[i]
                    +depreciation_cost[i]))*tax_rate  for i in range(len(operational_cost))]

        tax_cost[0]=0
        tax_cost[1]=0
        #for i in range(len(total_revenue)):
        cash_flow=[total_revenue[i]-(capital_cost_yr[i]
                +operational_cost[i]+labour_cost[i]+
                maintenance_cost[i]+emission_cost[i]+tax_cost[i]) for i in range(len(total_revenue))]
        NPV=np.npv(interest_rate,cash_flow)
        IRR=np.irr(cash_flow)
    #plt.bar(Years,cash_flow)

    npv=npf.npv(interest_rate,cash_flow)
    irr=npf.irr(cash_flow)
    #return(NPV_methane_pyrolysis,IRR_system)
    return [npv,irr,cash_flow]



In [15]:
###Local sensitivity analysis


## Generation of input variables for one at a time sensitivity analysis


plant_life=20 # years 
tax_rate=0.25 # percentage
interest_rate=0.10 # percentage

electricity_cost=56.12 #USD/MWh # Germany
iron_ore_cost=90 #usd/ton
emission_cost=30# USD 30 per ton 
carbon_steel_price=700#USD/ton

emission_factor=0.413# corresponsding to german emission factor
carbon_price=200
O2_price=40 #USD/ton
el_spec=50 # KWh/kgh2
h2_investment_2020=0.6 # Million euros/MWh2 
# Assume that 60% of the carbon produced could be sold




baseline=[tax_rate,interest_rate, electricity_cost,iron_ore_cost,
            emission_cost,carbon_steel_price,O2_price,el_spec,
                         h2_investment_2020]
baseline_npv=electrolyser_npv(*baseline)[0]
baseline_irr=electrolyser_npv(*baseline)[1]
baseline_npv

/home/abhinav/[/home/abhinav/anaconda_new]/lib/python3.7/site-packages/ipykernel_launcher.py:408: DeprecationWarning: numpy.npv is deprecated and will be removed from NumPy 1.20. Use numpy_financial.npv instead (https://pypi.org/project/numpy-financial/).
/home/abhinav/[/home/abhinav/anaconda_new]/lib/python3.7/site-packages/ipykernel_launcher.py:409: DeprecationWarning: numpy.irr is deprecated and will be removed from NumPy 1.20. Use numpy_financial.irr instead (https://pypi.org/project/numpy-financial/).


-1489.6955787999273

In [21]:
   
def pyrolysis_npv(tax_rate,interest_rate,electricity_cost,
                     iron_ore_cost,emission_cost,carbon_steel_price,carbon_price,ng_price):
    import import_ipynb
    import enthalpy_functions
    lf_rkt=10
    lf_dri_eaf=2
    carbon_sold=0.6
    emission_factor=0.413
    steel_prod_yr=3078060
    
    mol_weight_fe2o3=159.688 #in grams 
    mol_weight_Sio2=60.0843 #in grams
    mol_weight_al2o3=101.9613 #in grams
    mol_weight_feo=71.844 #in grams
    mol_weight_H2=2.01588 #in grams
    mol_weight_CH4=16.04 # in grams
    mol_weight_C=12.017 # in grams
    mol_weight_H2O=18.0153 #in grams
    mol_weight_cao=56.077#in grams
    mol_weight_mgo=40.3044 #in grams
    mol_weight_al2o3=101.9613 #in grams
    mol_weight_tin=118.07

    methane_ems=2.42 # kgco2/kgch4 by considering 1 MMBTU=1055.055 MJ and calorific value of CH4=48 MJ/kg
    #amount of CO2 released by burning 1 kg of CH4
    #Value for Germany taken from : 
    ##https://www.carbonfootprint.com/docs/2018_8_electricity_factors_august_2018_-_online_sources.pdf
    grid_emission_factor=0.413#ton/MWh
    ng_cal=48000 #ref https://www.jica.go.jp/english/our_work/climate_change/c8h0vm0000f7klc7-att/Appendix_E.pdf
    # Kj/kg calorific value of natural gas 
    ng_upstream_emission=17.2 #gCo2/MJ
    ng_up_em_kg=(ng_upstream_emission/ng_cal)*1000 #kgCO2/kgCH4
    gas_heater_eta=0.85  # Efficiency of the gas fired heater
    eta_el=0.9 # Efficiency of the electric heater used for the reactor 
    el_heater_cost=50 # Million USD
    tinprice=18000 #USD/T

    ss_steel_cost=2500 #USD/ton
    refractory_cost=800 #USD/ton
    steel_density=7700 #kg/m3
    refractory_density=2970 #kg/m3
    tank_manufacturing=1000 #USD/ton
    refractory_labour=1000 #USD/ton
    #Economic assumptions
    lf_reac=10
    lf_dri=2


    # Metallization rates considered for the DRI reduction shaft furnace
    alpha=0.94 
    # Percentage of hematite in the raw material. 
    fe2O3_pure=0.95 
    # percentage of iron in one mole of iron ore 
    Fe_O_ratio=((2*mol_weight_fe)/mol_weight_fe2o3)
    # kg of steel produced in the DRI-EAF system (1 ton)
    m3=1
    # Weight of total iron ore fed to the DRI-EAF system
    m1=m3/(fe2O3_pure*Fe_O_ratio*alpha)
    T1=298 #pellet temperature
    # Depends on the slag content , metallization rate 
    m2_feo=(m1*fe2O3_pure*(1-alpha)*Fe_O_ratio) 
    #print("FeO percentage :",m4_feo)
    # Percentage of silica in the ore pellets
    SIO2_per=0.03 
    #Percentage of alumina in the ore pellets
    Al2O3_per=0.02 
    m1_sio2=(m1*SIO2_per)# SiO2 weight in ton
    m1_al2o3=(m1*Al2O3_per) # weight in ton
    m2_fe=(m1-(m1_sio2+m1_al2o3+m2_feo))*Fe_O_ratio

    # to produce 1 mole of iron 2/3 rd mole of hydrogen are required
    H2_per_mol=3/2 
    # Weight of H2 for producing one mole of Fe in gms
    H2_weight=(H2_per_mol)*mol_weight_H2 
    # Number of moles in 1 ton of Fe
    mol_per_ton_fe=(1000000)/mol_weight_fe
    # Weight of hydrogen in kg to reduce 1 ton of Fe2O3
    h2_per_ton_steel=(H2_weight*mol_per_ton_fe)/1000



    h2_prod_yr=200000000 #in kg per year 

    # Plant is operational 95% of the time in an year 
    operating_hours=365*24*0.95 

    h2_prod_hr=h2_prod_yr/operating_hours


    #Excess hydrogen fed in the dri shaft furnace for complete reduction of iron ore 
    lambda_h2=1.20

    # Calculating the actual amount of hydrogen required for production of one ton of steel
    h2_per_ton_actual=lambda_h2*h2_per_ton_steel
    m4=h2_per_ton_actual # mass flow rate of hydrogen entering the shaft furnace

    steel_per_hr=h2_prod_hr/h2_per_ton_actual
    steel_prod_yr=steel_per_hr*operating_hours

    carbon_per_hr=(h2_prod_hr*3) # based on stoichiometry
    carbon_per_year=carbon_per_hr*operating_hours

    T_reactor=1200 #in C

    conversion_meth=0.90

    met_feed_rate=1837 #mol/sec

    #Multiplied by the molecular weight of methane
    met_feed_rate_kg=met_feed_rate*0.016 


    met_feed_rate_kg_hr=met_feed_rate_kg*(3600) 
    hydrogen_per_hr=(met_feed_rate_kg_hr*conversion_meth)/4



    #Reactor diameter
    dia_reac=5.21

    len_reac=8.66


    vol_reac=(3.14*dia_reac*dia_reac*len_reac)/4
    sur_area_reac=((2*np.pi*(dia_reac/2)) * len_reac) + ((np.pi*(dia_reac/2)**2)*2)

    p_reac_bot=19.1
    p_reac_top=15.3

    gas_holdup_bot=0.22
    gas_holdup_top=0.32
    gas_holdup_avg=0.294



    v_metal=(vol_reac*(1-gas_holdup_avg))

    t=(T_reactor+273.15)
    H_hydrogen_si_perkg=H2_enthalpy_2(t)*1000    
    # hydrogen demand per ton steel =m3
    # convert kJ to KWh by dividing the value with 3600
    H_hydrogen_tls=(H_hydrogen_si_perkg*m4)/3600


    t1=1173 # in Kelvin, temperature of the input hydrogen stream

    H_hydrogen_recovered_tls=((H2_enthalpy_2(t)-H2_enthalpy_2(t1))*m4)/3.6  
    ch4_inlet_temp=[]
    h_methane_inlet_enth=[]
    # find the inlet temperature of the methane stream entering the shaft furnace
    for i in range(298,973):

        #rint(i)
        h_methane_inlet=((CH4_enthalpy_1(i)*1000*met_feed_rate_kg_hr)/(3600*steel_per_hr))
        #print(h_methane_inlet)
        if h_methane_inlet<H_hydrogen_recovered_tls:
            ch4_inlet_temp.append(i)

        else:
            break


    H_methane_si_perkg=(CH4_enthalpy_2(t)-CH4_enthalpy_2(ch4_inlet_temp[-1]))*1000
    H_methane_total=(H_methane_si_perkg*met_feed_rate_kg_hr)/3600
    H_methane_tls=H_methane_total/steel_per_hr


    H_reac=74.5 #kj/mole
    H_reac_kg=(H_reac/mol_weight_CH4)*1000#Molecular weight of methane




    H_reac_total=(met_feed_rate_kg_hr*H_reac_kg*conversion_meth)/3600
    H_reac_tls=H_reac_total/steel_per_hr    
    H_reac_tls

    ### Carbon production from  methane pyrolysis

    # based on the reaction for every 1 kg of H2, 3 kg of carbon is produced
    carbon_per_hr=hydrogen_per_hr*3 
    carbon_per_tls=carbon_per_hr/steel_per_hr
    # It is assumed that the carbon deposited at the top 
    #of the molten metal is at the same temperature as the reactor 


    H_carbon_si_perkg=C_enthalpy(t)*1000    
    H_carbon_total=(H_carbon_si_perkg*carbon_per_hr)/3600
    H_carbon_tls=H_carbon_total/steel_per_hr

    eta_el_lmbr=0.8
    H_per_ton_steel=(H_reac_tls+H_hydrogen_tls+H_carbon_tls-H_hydrogen_recovered_tls)/(1000*eta_el_lmbr)#MWh
    H_per_ton_hydrogen= H_per_ton_steel/(lambda_h2*54)

    total_ng_req_tls=(met_feed_rate_kg_hr/steel_per_hr)

    el_heater_cost=50

    T1_tin=505
    #Calculation of delta h in j/mol

    # for t in range of 298 to 505 K
    h1_tin=-5855.135+15.961*T1_tin+((18.8702/1000)*T1_tin**2)-(6.2472334*10**-6*T1_tin**3)-(123920/T1_tin) 

    h1_j_g=h1_tin/mol_weight_tin
    T2_tin=800
    #T in the range of 505 to 800 K
    h2_tin=9496.31+8.2590486*T2_tin+((16.814429/1000)*T2_tin*T2_tin)-(5.24626*10**-6*(1/T2_tin**3))-(2162488/T2_tin)

    T3_tin=T_reactor
    h3_tin=[]
    # T in the range of 800 to 3000 K


    h3_tin=-1285.372+28.4512*T3_tin

    H_kj_kg=h3_tin/mol_weight_tin

    #### Heat required for heating tin inside the bubble column reactor from 298 K to T_final
    H_tin=H_kj_kg


    ### Density variation of liquid tin with changing temperature

    #### Reference : https://sci-hub.se/https://doi.org/10.1063/1.3467496

    c3=6979 #kg/m3
    c4=0.652 
    t_ref=505.08


    rho_final=c3-c4*(T_reactor-t_ref)
    ### Calculation of weight of liquid tin 

    weight_tin=rho_final*v_metal
    weight_tin/1000

    ### Capital cost of the metal
    #### Source : https://tradingeconomics.com/commodity/tin


    cost_perkg=tinprice/1000 # Converted to Euros 


    tin_cost=weight_tin*cost_perkg
    #Cost of tin in million Euros
    million=10**6
    tin_cost_million=tin_cost/million


    ## Enthalpy of hydrogen entering the shaft furnace
    dri_temp=900
    T4=(dri_temp+273.15)# k

    if T4<1000:
        h4=(H2_enthalpy_1(T4)*m4*1000) #kj/kg
        h4_kwh=(h4/3600)
    elif T4>1000:
        h4=(H2_enthalpy_2(T4)*m4*1000) #kj/kg 
        h4_kwh=(h4/3600)


    ## Mass flow rate of waste gases from the shaft furnace 

    # Mass flow rate of water in the waste stream per ton of steel production

    water_tls=((3*mol_weight_H2O)/(2*mol_weight_fe))*1000
    #print("Amount of water in the waste stream in kg/tls",water_tls)
    ## Calculation of the hydrogen in the waste stream 
    m5_h2=(h2_per_ton_steel*(lambda_h2-1)) #Kg/tls
    m5_h2o=water_tls
    m5=(m5_h2+m5_h2o)
    # The value varies with input flow rate of hydrogen inside the shaft furnace
    #print("Amount of hydrogen in the waste stream per ton of steel production in kg/tls",m5_h2,m5_h2o)

    ## Exhaust stream enthalpy

    """Temperature of the exhaust stream could vary according to the input
    temperature and flow rate of the hydrogen stream.
    Plant data from Syngas based DRI shaft furnace show that the temperature could vary from 275 C to 400 C.
    Higher temperatures are witnessed when the content of CO in the Syngas is higher.
    The reduction reaction between heamtite and CO is exothermic.
    On the other hand , hyrogen reduction of hematite is endothermic.
    Assumed temperature of 300 C for the enthalpy calculation."""


    # K Temperature of the exhaust gas leaving the reduction shaft furnace
    T5=573

    ### Energy contained in water stream


    h5_h2o=(m5_h2o*H2O_enthalpy(T5)*1000) #kJ/tls
    h5_h2=(H2_enthalpy_1(T5)*m5_h2*1000) #kJ/tls
    h5=(h5_h2o+h5_h2)
    h5_kwh=(h5/3600)


    ## Reaction enthalpy calculation
    #The reaction enthalpy has been taken as 99.5 kJ/mol based on stoichiometric calculations.

    h_activation=35 # kJ/mol activation enthalpy
    h_endothermic=99.5 # reaction enthalpy kJ/mol
    #Adding the activation enthalpy to the reaction enthalpy
    # value in kJ/mol
    h_reaction=h_activation+h_endothermic
    # Amount of heat to be supplied in kJ for production of 1 ton of molten steel
    h_reac=(h_reaction*m1*1000*fe2O3_pure)/mol_weight_fe2o3


    ## Energy contained in the solid stream exiting the DRI

    """Enthalpy of the residual FeO in the slag stream : https://webbook.nist.gov/cgi/cbook.cgi?ID=C1345251&Mask=2"""

    # The constants are used for a temperature range of 298-1650 K 
    # Exit temperature of the FeO stream contained in the solid stream exiting the reactor

    T2=873


    h2=((fe_enthalpy(T2)*m2_fe+sio2_enthalpy(T2)*m1_sio2+
        Al2O3_enthalpy(T2)*m1_al2o3+feo_enthalpy(T2)*m2_feo)*1000)*1000

    h2_kwh=h2/3600

  

    ## Energy balance of the shaft furnace


    Q_dri=((h5+h2)-h4)/3600

  

    ## Electric arc furnace mass and energy balance 

    ## input and tap temperatures

    T3=1923 # Tap temperature of the molten steel 1650 C

    ## Energy required to melt Fe in the EAF

    # Reference : NIST https://webbook.nist.gov/cgi/cbook.cgi?ID=C7439896&Mask=2&Type=JANAFS&Table=on
    Hfe_melting=247 # Melting enthalpy of steel per ton in Kj/kg
    #https://materialsdata.nist.gov/bitstream/handle/11115/166/Thermophysical%20Properties.pdf?sequence=3
    Hfe_T2=fe_enthalpy(T2) # Enthalpy of DRI at the input 
    Hfe_T3=fe_enthalpy_2(T3) # Enthalpy of molten steel at the output
    h3=(((Hfe_T3-Hfe_T2)*m2_fe*1000)+(m2_fe*Hfe_melting))*1000#kj/kg
    h3_kwh=h3/3600 #kwh/tls
  


    eta_el=0.60 # Efficiency of the transformer, arc 
    #, heat transfer, cooling losses, waste gas stream taken into consideration
    # The efficincy is considered lower to account for the loss of energy from the
    #scrap stream, the use of slag formers etc
    el_eaf=(h3_kwh)/eta_el



    # Specific energy consumption of HDRI-EAF system

    if Q_dri>0:
        EL_total=(el_eaf+(Q_dri/3600))
    else:
        EL_total=(el_eaf)
    EL_total_MWh=EL_total/1000
    print("Specific energy consumption of HDRI-EAF in MWh/tls",EL_total_MWh)




    ng_energy_feedstock=(total_ng_req_tls*48)/3.6

    ## Total energy consumption of Methane pyrolysis based HDRI-EAF system



    E_total_MWh=H_per_ton_steel+EL_total_MWh


    ## CO2 emission

    #For this investigation,
    #CO2 emissions for the natural gas supply chain are assumed to sum
    #up to 17.2 gCO2/MJ1 considering emissions for natural gas production,
    #long distance transport and distribution in a high pressure system

    # Emissions from hydrogen production
    # can be divided into direct and indirect emissions 
    emission_factor=0.413
    reac_ems_dr=(total_ng_req_tls*ng_up_em_kg)/1000 # upstream emissions are considered
    reac_ems_indr=H_per_ton_steel*emission_factor #emission factor in tco2/MWh


    EAF_co2=0.050 #ton/tls from exhaust gases 
    cao_emissions=0.056 #tco2/tls calcination of caco3
    co2_eaf_electrode=0.0070 #tco2/tls use of electrodes
    pellet_production=0.12#tco2/tls # upstream emissions from pellet production

    hdri_eaf_ems_dr=(EAF_co2+cao_emissions+co2_eaf_electrode+pellet_production)#tco2/tls
    hdri_eaf_ems_indr =(EL_total_MWh*emission_factor) #tco2/tls


    total_emission=(reac_ems_dr+reac_ems_indr+hdri_eaf_ems_dr+hdri_eaf_ems_indr)

    total_emission_cost=((hdri_eaf_ems_dr+reac_ems_dr)*emission_cost*steel_prod_yr)/10**6 # in Million USD

    def total_capital_cost(lang_factor_reac,lang_factor_dri_eaf):
    #Cost of tin per kg 
    ### Capital cost of the metal
    #### Source : https://tradingeconomics.com/commodity/tin
        cost_perkg=tinprice/1000 # Converted to Euros 


        tin_cost=weight_tin*cost_perkg
        #Cost of tin in million Euros
        million=10**6
        tin_cost_million=tin_cost/million


        #### Cost of the bubble column reactor

        """Considering that the reactor is made of 120 mm thick carbon steel. 
        The cost of stainless steel is assumed to be USD 2500 /ton. 
        The reactor is lined with refractory (magnesium oxide) layer of 600 mm 
        and the cost of refractory bricks is 800 USD/ton."""



        vol_steel_reactor=np.pi*((dia_reac+0.120/2)**2-(dia_reac/2)**2)*len_reac
        vol_refractory=np.pi*((dia_reac/2+0.6)**2-(dia_reac/2)**2)*len_reac
        weight_steel_reactor=(vol_steel_reactor*steel_density)/1000 # conversion to tons of steel
        weight_refractory=(vol_refractory*refractory_density)/1000 # conversion to tons 


        reactor_cost=((weight_steel_reactor*ss_steel_cost)+
                             (weight_refractory*refractory_cost)+(weight_steel_reactor*10000)+
                             (weight_refractory*1000))/10**6+(tin_cost_million)
        EAF_cost_per_ton_yr=140
            # The value includes installation costs 
            #https://iea-etsap.org/E-TechDS/PDF/I02-Iron&Steel-GS-AD-gct.pdf
        EAF_total_cost=(EAF_cost_per_ton_yr*steel_prod_yr)/10**6
            #### DRI Cost 
            #Link : https://www2.deloitte.com/content/dam/Deloitte/global/Documents/Manufacturing/gx_remaking%20the%20global%20steel%20industry_06_13.pdf

        DRI_cost_per_ton_yr=240 #USD/ton/per year
        # Cost of new DRI plant with 2 MT/annum =5.5 million euros or 6 million USD
         # A reduction in cost is considered as larger plants have smaller costs. (0.6)
        # The plant costs include all the costs including ISBL, OSBL, area development etc 
        # no Lang factor multiplication is required for such a plant
        #https://www.voestalpine.com/group/en/media/press-releases/2013-07-04-
         #voestalpine-entrusts-construction-of-the-direct-reduction-plant-in-texas-to-siemens-and-midrex/
        DRI_total_cost=(DRI_cost_per_ton_yr*steel_prod_yr)/10**6
        #### Pressure swing adsorber 

        PSA_cost=25 # Million USD 

        #### Gas fired metal furnace 
        #https://www.alibaba.com/showroom/gas-melting-furnace.html


        #lang_factor_reac=10
        #lang_factor_dri_eaf=6

        total_capital_cost=((((reactor_cost+PSA_cost+el_heater_cost)*lang_factor_reac)
                             +(EAF_total_cost+DRI_total_cost)))*lang_factor_dri_eaf
        #capital_cost={'EAF':EAF_total_cost,
                #'HDRI':DRI_total_cost,'Reactor':reactor_cost
                #,'PSA':PSA_cost,
                #'Electrical heater':el_heater_cost,
                #'Total capital cost':total_capital_cost}
        #df_capital_cost=pd.DataFrame.from_dict(capital_cost)
        return total_capital_cost


    def operational_cost(ng_price,electricity_cost,iron_ore_cost):    
        eaf_op_cost_tls=32 #t/yr of dri
        dri_op_cost_tls=13#t/yr of dri

        ng_cost=(total_ng_req_tls/20)*ng_price
        iron_ore_cost_tls=(m1)*iron_ore_cost
        electricity_cost_tls=(electricity_cost*E_total_MWh)

        operational_cost_annual=(((ng_cost+iron_ore_cost_tls+electricity_cost_tls+eaf_op_cost_tls+
                                   dri_op_cost_tls)*steel_prod_yr)/10**6)
        system_costs={'Natural gas':ng_cost,'Iron ore':iron_ore_cost_tls,'Electricity_cost':electricity_cost_tls,
                      'Total system costs':operational_cost_annual}
        return(operational_cost_annual)

    #Maintenance cost
    maintenance_cost_percentage=0.05
    labour_cost_tls=40 #USD/tls
    # 10% of the capital cost


    maintenance_cost_yr=(maintenance_cost_percentage*total_capital_cost(lf_reac,lf_dri))
    labour_cost_yr=(labour_cost_tls*steel_prod_yr)/10**6

    # Depreciation cost

    depreciation_yr=total_capital_cost(lf_reac,lf_dri)/plant_life

    ## Base case 
    def lcop(ng_price=6.58,electricity_cost=56.12,iron_ore_cost=90):
        i=interest_rate
        f_an=(i*((1+i)**plant_life)/((1+i)**plant_life-1))
        eac=(total_capital_cost(lf_reac,lf_dri)*f_an)+operational_cost(ng_price,electricity_cost,iron_ore_cost)+maintenance_cost_yr+labour_cost_yr+total_emission_cost
        lcop=eac/(steel_prod_yr/10**6)
        return lcop

       # Revenue
        # USD/ton


    revenue_steel=((carbon_steel_price*steel_prod_yr)/10**6)
        #USD/ton


        # Revenue generated in Million USD /year 

    revenue_carbon=(carbon_price*carbon_per_year*carbon_sold)/10**9


    total_revenue_usd_million=revenue_steel+revenue_carbon

    

    
   
    #reactor cost calculation
    #plant_life=25
    #interest_rate=0.10
    #tax_rate=0.35
    ## Emission calculations 
    """Institute for Global Environmental Strategies (2020). List of Grid Emission Factors, version 10.8. Available at: https://pub.iges.or.jp/pub/iges-list-grid-emission-factors
    All copyrights are reserved. The source must be clearly stated when this database is reproduced or transmitted in any form or by any means.

    Value for Germany taken from : https://www.carbonfootprint.com/docs/2018_8_electricity_factors_august_2018_-_online_sources.pdf"""








    #For this investigation,
    #CO2 emissions for the natural gas supply chain are assumed to sum
    #up to 17.2 gCO2/MJ1 considering emissions for natural gas production,
    #long distance transport and distribution in a high pressure system

    # Emissions from hydrogen production
    # can be divided into direct and indirect emissions 
    reac_ems_dr=(total_ng_req_tls*ng_up_em_kg)/1000 # upstream emissions are considered
    reac_ems_indr=H_per_ton_steel*emission_factor #emission factor in tco2/MWh


    EAF_co2=0.050 #ton/tls from exhaust gases 
    cao_emissions=0.056 #tco2/tls calcination of caco3
    co2_eaf_electrode=0.0070 #tco2/tls use of electrodes
    pellet_production=0.12#tco2/tls # upstream emissions from pellet production

    hdri_eaf_ems_dr=(EAF_co2+cao_emissions+co2_eaf_electrode+pellet_production)#tco2/tls
    hdri_eaf_ems_indr =(EL_total_MWh*emission_factor) #tco2/tls


    total_emission=(reac_ems_dr+reac_ems_indr+hdri_eaf_ems_dr+hdri_eaf_ems_indr)

     #Emission cost
     # The emissions costs are associated with direct emissions within the system boundar and not with
    #the indirect emissions from the use of electricity.  
    total_emission_cost=((hdri_eaf_ems_dr+reac_ems_dr)*emission_cost*steel_prod_yr)/10**6 # in Million USD







    #total_capital_cost=total_capital_cost(lf_rkt,lf_dri_eaf) 

    #operational_cost_annual=operational_cost(ng_price,electricity_cost)



    maintenance_cost_yr=(maintenance_cost_percentage*total_capital_cost(lf_rkt,lf_dri_eaf))
    labour_cost_yr=(labour_cost_tls*steel_prod_yr)/10**6





    # Depreciation cost

    depreciation_yr=total_capital_cost(lf_rkt,lf_dri_eaf)/plant_life

    ## Revenue

       # Revenue
        # USD/ton


    revenue_steel=((carbon_steel_price*steel_prod_yr)/10**6)
        #USD/ton


        # Revenue generated in Million USD /year 

    revenue_carbon=(carbon_price*carbon_per_year*carbon_sold)/10**9


    total_revenue_usd_million=revenue_steel+revenue_carbon


    years=np.arange(0,plant_life,1)
    Years=np.round(years,0)
    production_steel=np.repeat(steel_prod_yr/10**6,plant_life).tolist()
    #production_hydrogen=np.repeat(h2_prod_yr/1000000,plant_life).tolist()






    capital_cost_yr=np.repeat(0,plant_life).tolist()
    capital_cost_yr[0]=(total_capital_cost(lf_rkt,lf_dri_eaf)/3)
    capital_cost_yr[1]=(total_capital_cost(lf_rkt,lf_dri_eaf))*(2/3)
    operational_cost_yr=np.repeat(operational_cost(ng_price,electricity_cost,iron_ore_cost),plant_life).tolist()
    maintenance_cost=np.repeat(maintenance_cost_yr,plant_life).tolist()
    labour_cost=np.repeat(labour_cost_yr,plant_life).tolist()
    emission_cost=np.repeat(total_emission_cost,plant_life).tolist()
    depreciation_cost=np.repeat(depreciation_yr,plant_life).tolist()
    total_revenue=np.repeat(total_revenue_usd_million,plant_life).tolist()

    for i in range(2):
        production_steel[i]=0
        operational_cost_yr[i]=0
        labour_cost[i]=0
        maintenance_cost[i]=0
        emission_cost[i]=0 
        depreciation_cost[i]=0
        total_revenue[i]=0
    # tax calculation

    tax_cost=[(total_revenue[i]-(capital_cost_yr[i]+operational_cost_yr[i]+labour_cost[i]
                                           +maintenance_cost[i]+
                                 emission_cost[i]+
                                 depreciation_cost[i]))*tax_rate for i in range(plant_life)]
    tax_cost[0]=0
    tax_cost[1]=0

    cash_flow=[total_revenue[i]-(capital_cost_yr[i]+
                                           operational_cost_yr[i]+labour_cost[i]
                                           +maintenance_cost[i]
                                 +emission_cost[i]+tax_cost[i])for i in range(plant_life)]
    npv=npf.npv(interest_rate,cash_flow)
    irr=npf.irr(cash_flow)
    #return(NPV_methane_pyrolysis,IRR_system)
    #financial_parameters=npv
    return [npv,irr,cash_flow]




Specific energy consumption of HDRI-EAF in MWh/tls 0.5281038438147679
Specific energy consumption of HDRI-EAF in MWh/tls 0.5281038438147679


-524.4950859214756

In [29]:
def energiron(tax_rate,interest_rate,steel_prod_yr,emission_factor,electricity_cost,
                 iron_ore_cost,emission_cost,carbon_steel_price,ng_price):
    plant_life=20
    
    
    iron_ore_dri=1.4 #t/t dri
    metallization=0.94
    oxygen_dri=55 #NM3/tls
    iron_ore_tls=iron_ore_dri/metallization
    natural_gas=2.22 # gcal/tls
    ng_cal=48000 #kj/kg
    natural_gas_kg=(2.22*10**6*4.184)/ng_cal #conver gcal to Kj 
    
    el_eaf=0.528 #KWh/tls
    
    
    
    #emission_factor=0.5 #kgCO2/KWh
    ng_upstream_emission=17.2 #gCo2/MJ
    ng_emission_tls=(natural_gas_kg*ng_cal*ng_upstream_emission)/10**6 #convert KJ to MJ #kgco2/tls
    co2_selective_dri=235 #kg/tls
    co2_non_selective=306 #kg/tls
    el_dri=79 # KWh/tls
    total_el=(el_dri+el_eaf)/1000 # MWh/tls
    indirect_ems_energiron=total_el*emission_factor
    direct_ems_energiron=(co2_non_selective+co2_selective_dri)/1000+ng_emission_tls/1000
    total_emission= direct_ems_energiron+indirect_ems_energiron
    #### Economic parameters that could be included in the article
    EAF_cost_per_ton_yr=140
    # The value includes installation costs 
    #https://iea-etsap.org/E-TechDS/PDF/I02-Iron&Steel-GS-AD-gct.pdf
    EAF_total_cost=(EAF_cost_per_ton_yr*steel_prod_yr)/10**6
    #### DRI Cost 
    #Link : https://www2.deloitte.com/content/dam/Deloitte/global/Documents/Manufacturing/gx_remaking%20the%20global%20steel%20industry_06_13.pdf

    DRI_cost_per_ton_yr=240 #USD/ton/per year
    # Cost of new DRI plant with 2 MT/annum =5.5 million euros or 6 million USD
     # A reduction in cost is considered as larger plants have smaller costs. (0.6)
    # The plant costs include all the costs including ISBL, OSBL, area development etc 
    # no Lang factor multiplication is required for such a plant
    #https://www.voestalpine.com/group/en/media/press-releases/2013-07-04-
     #voestalpine-entrusts-construction-of-the-direct-reduction-plant-in-texas-to-siemens-and-midrex/
    DRI_total_cost=(DRI_cost_per_ton_yr*steel_prod_yr)/10**6
    #### Pressure swing adsorber 

    
    
    total_capital_cost=((EAF_total_cost+DRI_total_cost))*2
     ##Operational cost
    eaf_op_cost_tls=32 #t/yr of dri
    dri_op_cost_tls=13 #t/yr of dri
    ng_cost=((ng_price/21)*natural_gas_kg)# converting price/MMBTU to price/kg
    iron_ore_cost_tls=iron_ore_tls*iron_ore_cost
    electricity_cost_tls=total_el*electricity_cost
    operational_cost_annual=((ng_cost+iron_ore_cost_tls+electricity_cost_tls+
                              eaf_op_cost_tls+dri_op_cost_tls)*steel_prod_yr)/10**6
    #Maintenance cost
    maintenance_cost_percentage=0.10 
    labour_cost_tls=40 #USD/tls
    # 10% of the capital cost
    
    maintenance_cost_yr=((maintenance_cost_percentage*total_capital_cost))
    labour_cost_yr=((labour_cost_tls*steel_prod_yr)/10**6)
     #Emission cost
    total_emission_cost=((direct_ems_energiron*emission_cost*steel_prod_yr)/10**6) # in Million USD
    depreciation_yr=total_capital_cost/plant_life
     # Revenue
    # USD/ton
    total_revenue=(carbon_steel_price*steel_prod_yr)/10**6
     
    ## NPV and IRR calculations 
    years=np.arange(0,plant_life,1)
    Years=np.round(years,0)
    production_steel=np.repeat(steel_prod_yr/10**6,plant_life).tolist()
    #production_hydrogen=np.repeat(h2_prod_yr/1000000,plant_life).tolist()  
      
    capital_cost_yr=np.repeat(0,plant_life).tolist()
    capital_cost_yr[0]=(total_capital_cost/3)
    capital_cost_yr[1]=(total_capital_cost)*(2/3)
    operational_cost=np.repeat(operational_cost_annual,plant_life).tolist()
    maintenance_cost=np.repeat(maintenance_cost_yr,plant_life).tolist()
    labour_cost=np.repeat(labour_cost_yr,plant_life).tolist()
    emission_cost=np.repeat(total_emission_cost,plant_life).tolist()
    depreciation_cost=np.repeat(depreciation_yr,plant_life).tolist()
    total_revenue=np.repeat(total_revenue,plant_life).tolist()

    for i in range(2):
        production_steel[i]=0
        operational_cost[i]=0
        labour_cost[i]=0
        maintenance_cost[i]=0
        emission_cost[i]=0 
        depreciation_cost[i]=0
        total_revenue[i]=0
    # tax calculation
    
    
    tax_cost=[]
    for i in range(plant_life):
        tax_cost.append((total_revenue[i]-(capital_cost_yr[i]+operational_cost[i]+
                                           labour_cost[i]+maintenance_cost[i]
                                           +emission_cost[i]+depreciation_cost[i]))*tax_rate)
    for i in range(2):
        tax_cost[i]=0
    cash_flow=[]
    for i in range(plant_life):
        cash_flow.append(total_revenue[i]-(capital_cost_yr[i]+
                                           operational_cost[i]+labour_cost[i]
                                           +maintenance_cost[i]+emission_cost[i]+tax_cost[i]))
    npv=npf.npv(interest_rate,cash_flow)
    irr=npf.irr(cash_flow)
    #return(NPV_methane_pyrolysis,IRR_system)
    p=[npv,irr]
    q=cash_flow
    return p
  





In [150]:
plant_life=20

ng_price=6.58 #USD/MMBTU
iron_ore_cost=90 #usd/ton
electricity_cost=56.12 #USD/MWh # Germany
emission_cost=150# USD 30 per ton 
carbon_steel_price=650 #USD/ton
tax_rate=0.35
interest_rate=0.10
emission_factor=0.413 # corresponsding to german emission factor




In [151]:
baseline_energiron=[tax_rate,interest_rate,steel_prod_yr,emission_factor,electricity_cost,
                 iron_ore_cost,emission_cost,carbon_steel_price,ng_price]

In [152]:
baseline_energiron_npv=energiron(*baseline_energiron)[0]

baseline_energiron_npv

863.5809168470463

In [159]:
plant_life=20
ng_price=6.58 #USD/MMBTU
iron_ore_cost=90 #usd/ton
electricity_cost=56.12 #USD/MWh # Germany
emission_cost=150# USD 30 per ton 
carbon_steel_price=800#USD/ton
tax_rate=0.35
interest_rate=0.08
emission_factor=0.413 # corresponsding to german emission factor
carbon_price=200


In [160]:


baseline_pyrolysis=[tax_rate,interest_rate,electricity_cost,
                 iron_ore_cost,emission_cost,carbon_steel_price,carbon_price,ng_price]
baseline_mp_npv=pyrolysis_npv(*baseline_pyrolysis)[0]
baseline_mp_irr=pyrolysis_npv(*baseline_pyrolysis)[1]
baseline_mp_npv

Specific energy consumption of HDRI-EAF in MWh/tls 0.5281038438147679
Specific energy consumption of HDRI-EAF in MWh/tls 0.5281038438147679


796.0422802960514

In [174]:
plant_life=20

ng_price=6.58 #USD/MMBTU
iron_ore_cost=90 #usd/ton
electricity_cost=20 #USD/MWh 
emission_cost=150# USD 30 per ton 
carbon_steel_price=700#USD/ton
tax_rate=0.35
interest_rate=0.08
emission_factor=0.413 # corresponsding to german emission factor

O2_price=40 #USD/ton
el_spec=50# KWh/kgh2
h2_investment_2020=0.6# Million euros/MWh2 

In [175]:

baseline_electrolyser=[tax_rate,interest_rate, electricity_cost,iron_ore_cost,
            emission_cost,carbon_steel_price,O2_price,el_spec,
                         h2_investment_2020]
baseline_el_npv=electrolyser_npv(*baseline_electrolyser)[0]
baseline_el_irr=electrolyser_npv(*baseline_electrolyser)[1]
baseline_el_npv

/home/abhinav/[/home/abhinav/anaconda_new]/lib/python3.7/site-packages/ipykernel_launcher.py:408: DeprecationWarning: numpy.npv is deprecated and will be removed from NumPy 1.20. Use numpy_financial.npv instead (https://pypi.org/project/numpy-financial/).
/home/abhinav/[/home/abhinav/anaconda_new]/lib/python3.7/site-packages/ipykernel_launcher.py:409: DeprecationWarning: numpy.irr is deprecated and will be removed from NumPy 1.20. Use numpy_financial.irr instead (https://pypi.org/project/numpy-financial/).


844.4131063469462